In [ ]:
import sys
import pandas as pd
from lxml import etree
import re

print(sys.executable)

## Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [ ]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

In [ ]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference']]

df.iloc[1500:1510, :]

In [ ]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

In [ ]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

## Liage des articles de type commune

In [ ]:
# df des communes liées (pour décomptes et concaténation dans l’output final relu par OC et SN)
linked_communes_df = pd.DataFrame()

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[1500:1510, :]
#communes_df.shape

### Exact match method

#### With dpt

In [ ]:
# liage des communes avec dpt de localisation (dpt/exact match)
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df.shape

In [ ]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/exact match)
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on = 'NCCENR')
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# Test
communes_exact_nodpt_df
#linked_communes_df.shape
# note pour Margot : nunique() vs count() vs size, cf https://stackoverflow.com/questions/45759966/counting-unique-values-in-a-column-in-pandas-dataframe-like-in-qlik
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False, subset=['article_id']), :]

### Fuzzy method

#### With dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_communes_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_fuzzy_dpt_df]).drop_duplicates()



In [ ]:
# Tests
communes_fuzzy_dpt_df.head(20)
#linked_communes_df.shape
#linked_communes_df

#### Without dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_communes_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_fuzzy_nodpt_df]).drop_duplicates()



In [ ]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_communes_df.shape

In [ ]:
communes_fuzzy_nodpt_df.loc[communes_fuzzy_nodpt_df.duplicated(keep=False, subset=['article_id']), :]

## Liage des communes de localisation

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

### Exact match method

#### With dpt

In [ ]:
# liage des communes avec localisation (dpt/exact match)
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['localisationco','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, pas_communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape

In [ ]:
# des doublons de liage exact match dans un même dpt ?
#pas_communes_exact_dpt_df.article_id.duplicated().sum()
pas_communes_exact_dpt_df.loc[pas_communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/exact match)
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='localisationco',
                                   right_on = 'NCCENR')
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# Test
pas_communes_exact_nodpt_df
#linked_communes_df.shape

### Fuzzy match method

#### With dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_communes_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()



In [ ]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_communes_df.shape
#linked_communes_df

#### Without dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_communes_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()

In [ ]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_communes_df.shape

## Rapport et analyse

In [ ]:
print(linked_communes_df)

In [ ]:
# des doublons?
doublons_linked_communes_df = linked_communes_df.loc[linked_communes_df.duplicated(keep=False, subset=['article_id']), :]
print(doublons_linked_communes_df)


In [ ]:
report = dict(
    article = i,
    article_lies = len(linked_communes_df.index),
    article_lies_doublons = len(doublons_linked_communes_df.index),
    article_commune_exact = len(communes_exact_dpt_df.index),
    article_commune_nodpt_exact = len(communes_exact_nodpt_df.index),
    article_commune_fuzzy = len(communes_fuzzy_dpt_df.index),
    article_commune_nodpt_fuzzy = len(communes_fuzzy_nodpt_df.index),
    article_pas_commune_exact = len(pas_communes_exact_dpt_df.index),
    article_pas_commune_nodpt_exact = len(pas_communes_exact_nodpt_df.index),
    article_pas_commune_fuzzy = len(pas_communes_fuzzy_dpt_df.index),
    article_pas_commune_nodpt_fuzzy = len(pas_communes_fuzzy_nodpt_df.index),
    article_exactdpt = (linked_communes_df.method == 'dpt_exact').sum(),
    article_exactnodpt = (linked_communes_df.method == 'nodpt_exact').sum(),
    article_fuzzydpt = (linked_communes_df.method == 'dpt_fuzzy').sum(),
    article_fuzzynodpt = (linked_communes_df.method == 'nodpt_fuzzy').sum(),
)

# check
[print(k,':',v) for k, v in report.items()]
linked_communes_df.iloc[1500:1510, :]

In [ ]:
linked_communes_df['article_id'].nunique()

In [ ]:
# A REVOIR : lister les articles non liés et les append au df de liage final avec en valeur de method null
for col in linked_communes_df.columns:
    if col not in df.columns:
        df[col] = 'nulle'

print(df)

In [ ]:
new_rows_df = df[~df['article_id'].isin(linked_communes_df['article_id'])]

merged_df = linked_communes_df.append(new_rows_df, ignore_index=True)

merged_df

In [ ]:
articles_nuls = merged_df[merged_df['insee_code'] == 'nulle']

articles_nuls

In [ ]:
# des doublons?
merged_df_dupes = merged_df.loc[merged_df.duplicated(keep=False, subset=['article_id']), :]
print(merged_df_dupes)

In [ ]:
merged_df_duplicates = merged_df[merged_df.duplicated('article_id')]
print(merged_df_duplicates)

In [ ]:
merged_df_dupes.shape

In [ ]:
method_counts = merged_df['method'].value_counts()
print(method_counts)

## Cellule a revoir

Ici, ca bloque. J'essaie d'ordonner par vedette / localisationco si les valeurs sont les memes, pour faciliter le travail d'Olivier: malheureusement, j'avais reussi a le faire avec mon ancienne methode (quand je fusionnais deux dataframes ensemble), mais la je n'ai pas l'impression que ca fonctionne du tout.

In [ ]:
# Tri du dataframe par article_id
merged_df.sort_values('article_id', inplace=True)

# Création d'une colonne temporaire pour stocker les valeurs de 'vedette' et de 'localisationco' converties en chaînes de caractères
merged_df['vedette_str'] = merged_df['vedette'].astype(str)
merged_df['localisationco_str'] = merged_df['localisationco'].astype(str)

# Tri du dataframe en utilisant la colonne 'vedette_str' pour le tri principal et la colonne 'localisationco_str' pour le tri secondaire
merged_df.sort_values(['vedette_str', 'localisationco_str'], inplace=True)

# Suppression des colonnes temporaires
merged_df.drop(['vedette_str', 'localisationco_str'], axis=1, inplace=True)


In [ ]:
# Tri pour Margot
vedette_list = pd.unique(merged_df['vedette'])
merged_df['sort_key'] = merged_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
merged_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
merged_df = merged_df.drop('sort_key', axis=1)
# reorder (cf reference)
#merged_df = merged_df[['article_id', 'vedette']]
merged_df.to_csv('out/toto.csv', index=False)
merged_df

## Export

In [ ]:
# Sauvegarder le dataframe fusionné
merged_df.to_csv('../../utils/liage_po7.csv', index=False)
